In [15]:
import os 
import speech_recognition as sr 
from googletrans import Translator
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import time
from nltk.sentiment.vader import SentimentIntensityAnalyzer 
from flask import Flask, request, jsonify
from flask_cors import CORS
from pydub import AudioSegment  
from transformers import pipeline
import spacy
import moviepy.editor as mp 
from moviepy.editor import VideoFileClip  

In [16]:
app = Flask(__name__)
CORS(app, origins="http://localhost:3000", supports_credentials=True)

nlp = spacy.load("en_core_web_sm")

@app.route('/api/analyze-audio/<string:text>', methods=['GET'])
def analyze_text(text):
    if len(text) == 0:
        return jsonify({'error': 'No text provided'}), 400
    
    doc = nlp(text) 
    person_count = len([ent.text for ent in doc.ents if ent.label_ == "PERSON"]) 
    topics = [token.text for token in doc if token.is_alpha and not token.is_stop]
    
    return jsonify({'person_count' : person_count + 1 , 'topic' : topics[0], 'topic2' : topics[1]})
 

@app.route('/api/convert-video-to-mp3', methods=['POST'])
def convert_video_to_mp3():
    if 'file' not in request.files:
        return jsonify({'error': 'No file provided'}), 400

    video_file = request.files['file']
    if video_file.filename == '':
        return jsonify({'error': 'No selected file'}), 400

    if video_file: 
        try: 
            video_path = os.path.join('./uploads', video_file.filename)  
            print(video_path)
            video_file.save(video_path)
 
            audio_path = os.path.abspath('./uploads/output.mp3') 
            video_clip = VideoFileClip(video_path)
            audio_clip = video_clip.audio
            audio_clip.write_audiofile(audio_path)
            audio_clip.close()
            video_clip.close()

            return jsonify({'message': 'Video converted to MP3 successfully', 'audio_file': audio_path}), 200
        except Exception as e:
            return jsonify({'error': 'Conversion error', 'details': str(e)}), 500


def get_audio_length(audio_filename):
    audio = AudioSegment.from_file(audio_filename)
    return len(audio) / 1000  # Return length in seconds    

def split_audio(input_audio, output_dir, segment_length_ms=60000):
    print("Proccessing Input........") 
    audio = AudioSegment.from_file(input_audio) 
    total_length_ms = len(audio) 
    num_segments = total_length_ms // segment_length_ms 

    for i in range(num_segments):
        start_time = i * segment_length_ms
        end_time = (i + 1) * segment_length_ms
        segment = audio[start_time:end_time] 
        segment.export(os.path.join(output_dir, f"segment_{i}.wav"), format="wav")
        
    if total_length_ms % segment_length_ms > 0:
        start_time = num_segments * segment_length_ms
        end_time = total_length_ms
        last_segment = audio[start_time:end_time]
        last_segment.export(os.path.join(output_dir, f"segment_{num_segments}.wav"), format="wav")

def transcribe_audio_segments(segment_dir, lan):
    print("Converting to text........")
    recognizer = sr.Recognizer()
    
    transcribed_texts = []
    
    for filename in os.listdir(segment_dir): 
        audio_file = os.path.join(segment_dir, filename)
        
        with sr.AudioFile(audio_file) as source:
            audio_text = recognizer.record(source)
            text = recognizer.recognize_google(audio_text, language=lan)
            transcribed_texts.append(text)
            print(text)

    print(transcribed_texts)
    return transcribed_texts

def join_transcribed_texts(texts):
    return " ".join(texts)

def convert_mp3_to_wav(mp3_filename):
    wav_filename = mp3_filename.replace('.mp3', '.wav')
    audio = AudioSegment.from_mp3(mp3_filename)
    audio.export(wav_filename, format="wav")
    return wav_filename

@app.route('/api/convert-mp3-to-text', methods=['POST'])
def convert_mp3_to_text():
    if 'file' not in request.files:
        return jsonify({'error': 'No file part'}), 400

    file = request.files['file']
    language = request.form['language']
    
    if file.filename == '':
        return jsonify({'error': 'No selected file'}), 400

    if file:  
        mp3_filename = os.path.join('uploads', file.filename)
        file.save(mp3_filename)
        segment_dir = 'segments'  
        os.makedirs(segment_dir, exist_ok=True)
        audio_length = get_audio_length(mp3_filename)

        if audio_length > 60:
            split_audio(mp3_filename, segment_dir, segment_length_ms=60000)
            transcribed_texts = transcribe_audio_segments(segment_dir, language=language) 
            joined_text = join_transcribed_texts(transcribed_texts)
        else:
            recognizer = sr.Recognizer()

            wav_filename = convert_mp3_to_wav(mp3_filename)

            with sr.AudioFile(wav_filename) as source:
                audio_text = recognizer.record(source)
                text = recognizer.recognize_google(audio_text, language=language)
                joined_text = text
  
        joined_text = joined_text[:-82]        
        return jsonify({'text': joined_text})


@app.route('/api/convert-mp4-to-text', methods=['POST'])
def convert_mp4_to_text():
    language = request.form['language']

    file = os.path.abspath('./uploads/output.mp3') 

    if file: 
        mp3_filename = file
        wav_filename = mp3_filename.replace('.mp3', '.wav') 
        AudioSegment.from_mp3(mp3_filename).export(wav_filename, format="wav")

    segment_dir = 'segments'
    try:
        if not os.path.exists(segment_dir):
            os.mkdir(segment_dir)
    except Exception as e:
        print(f"Error creating directory: {e}")

    audio_length = get_audio_length(mp3_filename)

    if audio_length > 60: 
        split_audio(wav_filename, segment_dir, segment_length_ms=60000)  
        transcribed_texts = transcribe_audio_segments(segment_dir,lan=language) 
        print(transcribed_texts)
        joined_text = join_transcribed_texts(transcribed_texts) 
    else:
        recognizer = sr.Recognizer() 
        
        with sr.AudioFile(wav_filename) as source:
            audio_text = recognizer.record(source)
            text = recognizer.recognize_google(audio_text, language=language)
            joined_text = text

    return jsonify({'text': joined_text})


@app.route('/api/translate_toar/<string:text>', methods=['GET'], endpoint='translate_to_ar')
def translate_to_ar(text):    
    translator = Translator() 
    arabic_translation = translator.translate(text, src='auto', dest='ar').text
    return jsonify({'translated_txt': arabic_translation})


@app.route('/api/translate_totr/<string:text>', methods=['GET'], endpoint='translate_to_tr')
def translate_to_tr(text):  
    translator = Translator()    

    turkish_translation = translator.translate(text, src='auto', dest='tr').text
    return jsonify({'translated_txt': turkish_translation})

@app.route('/api/translate_toen/<string:text>', methods=['GET'], endpoint='translate_to_en')
def translate_to_en(text):  
    translator = Translator()    

    english_translation = translator.translate(text, src='auto', dest='en').text
    return jsonify({'translated_txt': english_translation})

@app.route('/api/translate_tohi/<string:text>', methods=['GET'], endpoint='translate_to_hi')
def translate_to_hi(text):  
    translator = Translator()    

    hindi_translation = translator.translate(text, src='auto', dest='hi').text
    return jsonify({'translated_txt': hindi_translation})

@app.route('/api/findtopic/<string:text>', methods=['GET'])
def topic_finder(text):
    
    pipe = pipeline("text-classification", model="unitary/toxic-bert")
    pipe2 = pipeline("summarization", model="google/pegasus-xsum")

    topic = pipe(text)  
    topic = topic[0]['label']

    topic2 = pipe2(text, max_length = 20)
    topic2 = topic2[0]['summary_text']
    return jsonify({'topic': topic, 'topic2': topic2})

@app.route('/api/findSummary/<string:text>', methods=['GET'])
def summary_find(text):
    pipe = pipeline("summarization", model="google/pegasus-xsum")
    output = pipe(text, max_length = 200)
    final_summary = output[0]['summary_text']

    translator = Translator() 
    arabic_summary = translator.translate(final_summary, src='en', dest='ar').text
    turkish_summary = translator.translate(final_summary, src='en', dest='tr').text

    return jsonify({'summary_en': final_summary, 'summary_ar': arabic_summary, 'summary_tr': turkish_summary})


@app.route('/api/sentiment/<string:text>', methods=['GET'])
def sentiment(text):
    nltk.download('vader_lexicon')
    analyzer = SentimentIntensityAnalyzer()
    scores = analyzer.polarity_scores(text)
    positive_percent = round(scores['pos'] * 100, 2)
    negative_percent = round(scores['neg'] * 100, 2)

    return jsonify({'positive': positive_percent, 'negative': negative_percent})

        
if __name__ == '__main__': 
    app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


./uploads\Arabic Conversation for Beginners #1 [Turn On Subtitles].mp4
MoviePy - Writing audio in c:\Projects\Audio_Analysis_UI_UX\uploads\output.mp3


127.0.0.1 - - [31/Dec/2023 18:10:21] "POST /api/convert-video-to-mp3 HTTP/1.1" 200 -


MoviePy - Done.
Proccessing Input........
Converting to text........
السلام عليكم كيف حالك يا اخي الحمد لله بخير كيف حالك انت يا اخي الحمد لله انا بخير ممكن ان نتعرف ممكن اسمي خاص ما اسمك اسم سيف الله اهلا بك اهلا بك من اين انت يا سيف الله انا من الشيشان ومن اين انت انا من تركيا وهل انت روسي لا انا لست روسيا انا شيشاني هل انت تركي لا انا لست تركيا انا شيشاني ايضا وكم عمرك 23 عاما وكم عمرك انت عمري 29 عاما ما شاء الله الحمد لله هل انت متزوج وهل انت متزوج ايضا الحمد لله انا متزوج ايضا هل لك اولاد وهل لك اولادك فقط اين تسكن يا اخي
شيشان في روزنه واين انت تسكن يا اخي انا اسكن في مدينه اسطنبول ومدينه جروزني مدينه كبيره او صغيره هي صغيره هل اسطنبول كبيره او صغيره مدينه اسطنبول كبيره جدا من اكبر المدن في العالم وهل تسكن في في مركز المدينه او مثلا في القريه انا اسكن في مركز المدينه ما شاء الله اين انت تسكن في اسطنبول انا اسكن في قريه وهل تسكن في بيت او في شقه في الشقه واين انت تسكن في البيت او في الشقه انا اسكن في بيت شقتك كبيره او صغيره شقتي كبيره كم غرفه فيها فيها فيها فيها فيها ثلاث غرف هذه

127.0.0.1 - - [31/Dec/2023 18:11:48] "POST /api/convert-mp4-to-text HTTP/1.1" 200 -


نوتيفيكيشن اباوت نيو فيديوز دوت كوم اوت اندلسي
['السلام عليكم كيف حالك يا اخي الحمد لله بخير كيف حالك انت يا اخي الحمد لله انا بخير ممكن ان نتعرف ممكن اسمي خاص ما اسمك اسم سيف الله اهلا بك اهلا بك من اين انت يا سيف الله انا من الشيشان ومن اين انت انا من تركيا وهل انت روسي لا انا لست روسيا انا شيشاني هل انت تركي لا انا لست تركيا انا شيشاني ايضا وكم عمرك 23 عاما وكم عمرك انت عمري 29 عاما ما شاء الله الحمد لله هل انت متزوج وهل انت متزوج ايضا الحمد لله انا متزوج ايضا هل لك اولاد وهل لك اولادك فقط اين تسكن يا اخي', 'شيشان في روزنه واين انت تسكن يا اخي انا اسكن في مدينه اسطنبول ومدينه جروزني مدينه كبيره او صغيره هي صغيره هل اسطنبول كبيره او صغيره مدينه اسطنبول كبيره جدا من اكبر المدن في العالم وهل تسكن في في مركز المدينه او مثلا في القريه انا اسكن في مركز المدينه ما شاء الله اين انت تسكن في اسطنبول انا اسكن في قريه وهل تسكن في بيت او في شقه في الشقه واين انت تسكن في البيت او في الشقه انا اسكن في بيت شقتك كبيره او صغيره شقتي كبيره كم غرفه فيها فيها فيها فيها فيها ثلاث غرف هذه ليست كبيره انها 

127.0.0.1 - - [31/Dec/2023 18:11:50] "GET /api/translate_toen/السلام%20عليكم%20كيف%20حالك%20يا%20اخي%20الحمد%20لله%20بخير%20كيف%20حالك%20انت%20يا%20اخي%20الحمد%20لله%20انا%20بخير%20ممكن%20ان%20نتعرف%20ممكن%20اسمي%20خاص%20ما%20اسمك%20اسم%20سيف%20الله%20اهلا%20بك%20اهلا%20بك%20من%20اين%20انت%20يا%20سيف%20الله%20انا%20من%20الشيشان%20ومن%20اين%20انت%20انا%20من%20تركيا%20وهل%20انت%20روسي%20لا%20انا%20لست%20روسيا%20انا%20شيشاني%20هل%20انت%20تركي%20لا%20انا%20لست%20تركيا%20انا%20شيشاني%20ايضا%20وكم%20عمرك%2023%20عاما%20وكم%20عمرك%20انت%20عمري%2029%20عاما%20ما%20شاء%20الله%20الحمد%20لله%20هل%20انت%20متزوج%20وهل%20انت%20متزوج%20ايضا%20الحمد%20لله%20انا%20متزوج%20ايضا%20هل%20لك%20اولاد%20وهل%20لك%20اولادك%20فقط%20اين%20تسكن%20يا%20اخي%20شيشان%20في%20روزنه%20واين%20انت%20تسكن%20يا%20اخي%20انا%20اسكن%20في%20مدينه%20اسطنبول%20ومدينه%20جروزني%20مدينه%20كبيره%20او%20صغيره%20هي%20صغيره%20هل%20اسطنبول%20كبيره%20او%20صغيره%20مدينه%20اسطنبول%20كبيره%20جدا%20من%20اكبر%20المدن%20في%20العالم%20وهل%20تسكن%20